In [1]:
import torch, torch.nn as nn, torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# دیتاست و DataLoader
transform = transforms.ToTensor()
train_loader = DataLoader(datasets.MNIST('.', train=True, download=True, transform=transform), batch_size=64, shuffle=True)
test_loader = DataLoader(datasets.MNIST('.', train=False, download=True, transform=transform), batch_size=1)

# مدل مینیمال CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, 1, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, 1, 1)
        self.fc1 = nn.Linear(32*7*7, 128)
        self.fc2 = nn.Linear(128, 10)
    def forward(self, x):
        x = torch.relu(self.pool(torch.relu(self.conv1(x))))
        x = torch.relu(self.pool(torch.relu(self.conv2(x))))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# آموزش
for epoch in range(5):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        loss = criterion(model(images), labels)
        loss.backward()
        optimizer.step()

# ارزیابی
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"دقت شبکه روی دیتاست تست: {100 * correct / total:.2f}%")


100.0%
100.0%
100.0%
100.0%


دقت شبکه روی دیتاست تست: 98.99%


In [2]:
# وارد کردن کتابخانه‌های ضروری
# torch: کتابخانه اصلی PyTorch برای محاسبات تنسور و شبکه عصبی
# nn: ماژول شبکه‌های عصبی PyTorch برای تعریف مدل‌ها و لایه‌ها
# optim: ماژول بهینه‌سازی برای بروزرسانی وزن‌ها
# datasets, transforms: برای بارگذاری و پیش‌پردازش دیتاست MNIST
# DataLoader: برای ساخت mini-batch و ترتیب‌دهی داده‌ها
import torch, torch.nn as nn, torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# =======================
# بارگذاری دیتاست MNIST
# transform=transforms.ToTensor(): تبدیل تصاویر PIL به Tensor و نرمالیزه کردن به [0,1]
# train=True/False: تعیین اینکه دیتاست آموزش یا تست بارگذاری شود
# batch_size: تعداد نمونه در هر mini-batch
# shuffle=True: داده‌ها قبل از هر epoch به صورت تصادفی جابجا شوند
train_loader = DataLoader(
    datasets.MNIST('.', train=True, download=True, transform=transforms.ToTensor()),
    batch_size=64, shuffle=True
)
test_loader = DataLoader(
    datasets.MNIST('.', train=False, download=True, transform=transforms.ToTensor()),
    batch_size=1
)

# =======================
# تعریف مدل CNN مینیمال
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        # conv1: لایه کانولوشن با 1 کانال ورودی، 16 کانال خروجی، کرنل 3x3، stride=1، padding=1
        # Conv2d اعمال فیلتر کانولوشن روی تصویر را انجام می‌دهد
        self.conv1 = nn.Conv2d(1,16,3,1,1)
        # conv2: لایه کانولوشن دوم با 16 کانال ورودی و 32 کانال خروجی
        self.conv2 = nn.Conv2d(16,32,3,1,1)
        # fc1: لایه کاملاً متصل که ابعاد 32*7*7 را به 128 نورون کاهش می‌دهد
        self.fc1 = nn.Linear(32*7*7,128)
        # fc2: لایه خروجی که 128 نورون را به 10 کلاس خروجی MNIST تبدیل می‌کند
        self.fc2 = nn.Linear(128,10)

    # تابع forward: مسیر عبور داده‌ها در مدل
    def forward(self,x):
        # اعمال کانولوشن اول + MaxPooling 2x2 + ReLU
        # nn.functional.max_pool2d: کاهش ابعاد ویژگی‌ها با MaxPooling
        # torch.relu: فعال‌سازی ReLU برای اضافه کردن غیرخطی بودن
        x = torch.relu(nn.functional.max_pool2d(self.conv1(x),2))
        # اعمال کانولوشن دوم + MaxPooling 2x2 + ReLU
        x = torch.relu(nn.functional.max_pool2d(self.conv2(x),2))
        # تبدیل تنسور به شکل (batch_size, flattened) برای لایه کاملاً متصل
        x = x.view(x.size(0),-1)
        # اعمال fc1 + ReLU
        x = torch.relu(self.fc1(x))
        # اعمال fc2 (خروجی نهایی) بدون فعال‌سازی چون CrossEntropyLoss خودش softmax را اعمال می‌کند
        return self.fc2(x)

# =======================
# تعیین device برای اجرا روی GPU در صورت وجود
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# انتقال مدل به device
model = CNN().to(device)

# =======================
# تعریف بهینه‌ساز Adam برای آپدیت وزن‌ها
# lr=0.001: نرخ یادگیری که مشخص می‌کند وزن‌ها چقدر در هر قدم تغییر کنند
opt = optim.Adam(model.parameters(),lr=0.001)

# تعریف تابع خطا CrossEntropyLoss برای دسته‌بندی چندکلاسه
# این تابع هم softmax اعمال می‌کند و هم log و loss نهایی را محاسبه می‌کند
loss_fn = nn.CrossEntropyLoss()

# =======================
# آموزش مدل
# هر batch شامل images و labels است
# optimizer.zero_grad(): پاک کردن گرادیان‌های قبلی قبل از backward
# loss_fn(model(images),labels).backward(): محاسبه گرادیان برای پارامترها
# opt.step(): آپدیت پارامترها با توجه به گرادیان
for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)
    opt.zero_grad()
    loss_fn(model(images),labels).backward()
    opt.step()

# =======================
# ارزیابی مدل روی دیتاست تست
# model.eval(): حالت ارزیابی (غیر آموزشی)
# torch.no_grad(): غیر فعال کردن محاسبه گرادیان برای صرفه‌جویی در حافظه
correct = 0
model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        # argmax(1): گرفتن کلاس با بیشترین احتمال پیش‌بینی شده
        correct += (model(images).argmax(1)==labels).sum().item()

# چاپ دقت نهایی مدل روی دیتاست تست
print("Accuracy:", correct/len(test_loader))


Accuracy: 0.9782
